#### Datansatz und Datenimport

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [3]:
df.reset_index(inplace= True)

In [5]:
df.head()

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo


#### **P1: Prüfen Sie, ob ein Zusammenhang zwischen dem Trinkgeldgeben und dem Vorhandensein eines der folgenden Produkte in einer Bestellung existiert: 24852, 24964, 2120**

In [13]:
df.groupby(["product_id"]).agg(tip_opp= ("tip", "count"), tips= ("tip", "sum")).reset_index()

,product_id,tip_opp,tips
0,1,303,182
1,2,18,9
2,3,96,58
3,4,66,21
4,5,9,6
...,...,...,...
45747,49684,4,2
45748,49685,13,3
45749,49686,33,8
45750,49687,2,1


In [15]:
df.loc[df.product_id.isin([24852, 24964, 2120])]

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
148,1726146,18,24964,29,1,56463,41,3,8,14.0,1,Organic Garlic,83,4,produce,fresh vegetables,Calaveras
239,3249426,22,24852,6,0,73241,1,0,15,NaN,0,Banana,24,4,produce,fresh fruits,Fresno
273,2769108,27,24852,3,1,130982,7,0,16,7.0,1,Banana,24,4,produce,fresh fruits,Plumas
333,1393512,34,24964,9,1,53965,25,5,9,4.0,1,Organic Garlic,83,4,produce,fresh vegetables,Calaveras
346,1393510,34,24852,1,1,53965,25,5,9,4.0,1,Banana,24,4,produce,fresh fruits,Calaveras
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132129,5411136,606861,24852,2,1,177763,3,1,15,4.0,0,Banana,24,4,produce,fresh fruits,Mariposa
6132193,1987718,606868,24852,10,1,187352,41,3,11,3.0,1,Banana,24,4,produce,fresh fruits,Glenn
6132294,3738097,606880,24852,1,1,20236,2,5,13,30.0,0,Banana,24,4,produce,fresh fruits,Yolo
6132434,4417394,606892,24852,14,1,113114,41,0,21,8.0,1,Banana,24,4,produce,fresh fruits,Merced


In [69]:
pd.crosstab(df.product_id.isin([24852, 24964, 2120]), df.tip, margins= True)

tip,0,1,All
product_id,,,
False,3335155,2686369,6021524
True,53167,57978,111145
All,3388322,2744347,6132669


In [26]:
pd.crosstab(df.product_id.isin([24852, 24964, 2120]), df.tip, margins= True, normalize= "index")

tip,0,1
product_id,,
False,0.553872,0.446128
True,0.478357,0.521643
All,0.552504,0.447496


In [59]:
df_tip1 = df.loc[df.product_id.isin([24852, 24964, 2120])][["product_id", "tip"]].reset_index(drop= True)
df_tip1.head()

,product_id,tip
0,24964,1
1,24852,0
2,24852,1
3,24964,1
4,24852,1


In [65]:
df_tip1.loc[df_tip1.tip == 1]["tip"].count() / df_tip1.tip.count()

0.5216428989158307

In [66]:
df_tip1.loc[df_tip1.tip == 0]["tip"].count() / df_tip1.tip.count()

0.47835710108416934

In [60]:
df_tip2 = df.loc[~df.product_id.isin([24852, 24964, 2120])][["product_id", "tip"]].reset_index(drop= True)
df_tip2.head()

,product_id,tip
0,13176,1
1,11109,1
2,10246,1
3,22035,1
4,49683,1


In [64]:
df_tip2.loc[df_tip2.tip == 1]["tip"].count() / df_tip2.tip.count()

0.446127757690578

In [63]:
df_tip2.loc[df_tip2.tip == 0]["tip"].count() / df_tip2.tip.count()

0.553872242309422

In [67]:
df_tip1.corr()

,product_id,tip
product_id,1.000000,-0.076068
tip,-0.076068,1.000000


In [68]:
df_tip2.corr()

,product_id,tip
product_id,1.000000,-0.000941
tip,-0.000941,1.000000


#### **P2**